In [4]:
import os
import json
import pandas as pd

In [6]:
test_id = "test_1660590398"
results_path = f"/Users/Vishvig/Desktop/Education/UoB/Courses/Dissertation/LV=/PoC2Prod/codebase/results/{test_id}"


In [106]:
runs = os.listdir(results_path)
print(runs)
runs.remove('.DS_Store')

['135', '307', '61', '95', '338', '300', '132', '59', '92', '66', '336', '104', '309', '50', '68', '103', '331', '57', '168', '391', '365', '157', '150', '362', '396', '159', '32', '166', '354', '192', '35', '195', '353', '161', '330', '102', '69', '56', '105', '337', '51', '308', '58', '133', '301', '67', '93', '306', '134', '94', '339', '60', '34', '160', '352', '194', '33', '158', '193', '355', '167', '397', '363', '151', '169', '156', '364', '390', '216', '229', '211', '227', '218', '.DS_Store', '220', '280', '274', '273', '287', '245', '289', '242', '221', '226', '219', '210', '217', '228', '288', '243', '244', '286', '272', '275', '281', '257', '268', '250', '292', '266', '259', '261', '295', '235', '232', '204', '203', '294', '260', '267', '293', '258', '251', '256', '269', '202', '205', '233', '234', '346', '174', '180', '379', '20', '187', '18', '173', '341', '27', '9', '0', '383', '145', '377', '11', '348', '370', '142', '384', '7', '29', '16', '189', '129', '42', '89', '116'

In [107]:
final_list = []

for run in runs:
    row = []
    base_path = os.path.join(results_path, run)
    test_details_path = os.path.join(base_path, 'test_run_details.json')
    statistics_path = os.path.join(base_path, 'out', 'statistics.json')
    node_util_metrics_path = os.path.join(base_path, 'metrics', 'node_utilization.csv')
    cluster_util_metrics_path = os.path.join(base_path, 'metrics', 'cluster_utilization.csv')    
    
    try:
        with open(test_details_path, 'r') as f:
            test_details = json.loads(f.read())
        with open(statistics_path, 'r') as f:
            statistics = json.loads(f.read())
        node_util_df = pd.read_csv(node_util_metrics_path, index_col=False)
        cluster_util_df = pd.read_csv(cluster_util_metrics_path, index_col=False)
    except FileNotFoundError:
        continue
    
    
    node_util_cpu_agg = node_util_df.agg({'node_cpu_utilization%': ['min', 'mean', 'max']})
    node_util_mem_agg = node_util_df.agg({'node_mem_utilization%': ['min', 'mean', 'max']})
    
    cluster_util_cpu_agg = cluster_util_df.agg({'cluster_cpu_utilization%': ['min', 'mean', 'max']})
    cluster_util_mem_agg = cluster_util_df.agg({'cluster_mem_utilization%': ['min', 'mean', 'max']})
    
    row.append(test_details['threads_info']['num_threads'])
    row.append(test_details["threads_info"]["ramp_time"])
    row.append(test_details["threads_info"]["loops"])
    row.append(test_details["instance_count"])
    row.append(statistics["HTTP Request"]["sampleCount"])
    
    row.append(test_details["resources"]["requests"]["cpu"])
    row.append(test_details["resources"]["requests"]["memory"])
    row.append(test_details["resources"]["limits"]["cpu"])
    row.append(test_details["resources"]["limits"]["memory"])
    
    
    row.append(node_util_cpu_agg["node_cpu_utilization%"]["min"])
    row.append(node_util_cpu_agg["node_cpu_utilization%"]["mean"])
    row.append(node_util_cpu_agg["node_cpu_utilization%"]["max"])
    row.append(node_util_mem_agg["node_mem_utilization%"]["min"])
    row.append(node_util_mem_agg["node_mem_utilization%"]["mean"])
    row.append(node_util_mem_agg["node_mem_utilization%"]["max"])
    
    row.append(cluster_util_cpu_agg["cluster_cpu_utilization%"]["min"])
    row.append(cluster_util_cpu_agg["cluster_cpu_utilization%"]["mean"])
    row.append(cluster_util_cpu_agg["cluster_cpu_utilization%"]["max"])
    row.append(cluster_util_mem_agg["cluster_mem_utilization%"]["min"])
    row.append(cluster_util_mem_agg["cluster_mem_utilization%"]["mean"])
    row.append(cluster_util_mem_agg["cluster_mem_utilization%"]["max"])
    
    row.append(statistics["HTTP Request"]["errorPct"])
    row.append(statistics["HTTP Request"]["minResTime"])
    row.append(statistics["HTTP Request"]["meanResTime"])
    row.append(statistics["HTTP Request"]["maxResTime"])
    row.append(statistics["HTTP Request"]["receivedKBytesPerSec"])
    row.append(statistics["HTTP Request"]["sentKBytesPerSec"])
    final_list.append(row)

In [108]:
df = pd.DataFrame(final_list, columns=['thread_per_loop', 'ramp_time', 'loops', 'instance_count', 'total_threads',
                                       'request_cpu', 'request_mem', 'limit_cpu', 'limit_mem',
                                       'min_per_node_cpu', 'avg_per_node_cpu', 'max_per_node_cpu', 'min_per_node_mem', 'avg_per_node_mem', 'max_per_node_mem',
                                       'min_cluster_cpu', 'avg_cluster_cpu', 'max_cluster_cpu', 'min_cluster_mem', 'avg_cluster_mem', 'max_cluster_mem',
                                       'error_rate', 'min_res_time', 'avg_res_time', 'max_res_time',
                                       "received_kbytes", "sent_kbytes"]
                 )
df



,thread_per_loop,ramp_time,loops,instance_count,total_threads,request_cpu,request_mem,limit_cpu,limit_mem,min_per_node_cpu,...,max_cluster_cpu,min_cluster_mem,avg_cluster_mem,max_cluster_mem,error_rate,min_res_time,avg_res_time,max_res_time,received_kbytes,sent_kbytes
0,250,5,2,1,500,1,2Gi,2,4Gi,0.008735,...,6.781319,0.180071,0.186881,0.214941,75.4,6.0,1299.694,7344.0,545.374227,27550.005873
1,40,1,5,2,200,500m,1Gi,1,2Gi,0.009014,...,23.003463,0.180048,0.306398,0.416207,66.5,8.0,336.290,1182.0,847.431356,32773.503074
2,200,1,1,1,200,500m,1Gi,1,2Gi,0.008735,...,3.134450,0.180179,0.186776,0.212863,94.5,193.0,794.670,1596.0,321.577603,62561.736272
3,20,10,10,1,200,2,2Gi,4,4Gi,0.008735,...,3.134450,0.180148,0.185834,0.214941,0.0,43.0,76.340,132.0,980.225336,13764.434288
4,200,30,1,2,200,2,2Gi,4,4Gi,0.009014,...,23.003463,0.179869,0.314572,0.416207,0.0,56.0,83.390,116.0,332.054843,4654.659555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,250,10,2,1,500,2,2Gi,4,4Gi,0.008735,...,6.781319,0.180071,0.186954,0.214941,52.0,5.0,488.682,1110.0,1076.589638,30299.491343
390,250,60,2,1,500,1,2Gi,2,4Gi,0.008735,...,17.284390,0.180071,0.188436,0.214941,30.0,5.0,171.502,1039.0,286.226665,5561.425341
391,500,1,2,1,1000,2,2Gi,4,4Gi,0.008735,...,17.284390,0.180071,0.189408,0.214941,80.7,6.0,9420.845,65423.0,152.761167,8629.788813
392,100,10,1,1,100,500m,1Gi,1,2Gi,0.008735,...,1.469943,0.180179,0.182713,0.212863,14.0,31.0,617.830,1148.0,405.162202,6605.617560


In [109]:
df = df[(df["error_rate"] <= 1) & (df["avg_res_time"] <= 500) & (df['max_res_time'] <= 1000)]
df.head()

,thread_per_loop,ramp_time,loops,instance_count,total_threads,request_cpu,request_mem,limit_cpu,limit_mem,min_per_node_cpu,...,max_cluster_cpu,min_cluster_mem,avg_cluster_mem,max_cluster_mem,error_rate,min_res_time,avg_res_time,max_res_time,received_kbytes,sent_kbytes
3,20,10,10,1,200,2,2Gi,4,4Gi,0.008735,...,3.134450,0.180148,0.185834,0.214941,0.0,43.0,76.340,132.0,980.225336,13764.434288
4,200,30,1,2,200,2,2Gi,4,4Gi,0.009014,...,23.003463,0.179869,0.314572,0.416207,0.0,56.0,83.390,116.0,332.054843,4654.659555
7,10,60,10,1,100,2,2Gi,4,4Gi,0.008735,...,3.134450,0.180179,0.186865,0.212863,0.0,42.0,63.020,103.0,90.469357,1270.380870
8,40,10,5,1,200,2,2Gi,4,4Gi,0.008735,...,3.134450,0.180148,0.185838,0.214941,0.0,49.0,90.010,161.0,985.407575,13837.203865
10,200,30,1,2,200,1,2Gi,2,4Gi,0.009014,...,23.003463,0.179869,0.313073,0.416207,0.0,58.0,82.235,113.0,332.054843,4654.659555


In [110]:
def cpu_float(cpu):
    """
    Convert all cpu values seen in K8s to milli-cpus.
    Milli-CPUs is taken because that is the standard of cpu requests and limits in K8s
    :param cpu: The CPU value to be converted
    :return: Milli-CPU float value
    """
    if 'u' in cpu:
        # Convert micro-cpu units to milli-cpus
        return float(cpu.rstrip('u')) / 10 ** 3
    elif 'n' in cpu:
        # Convert nano-cpu units to milli-cpus
        return float(cpu.rstrip('n')) / 10 ** 6
    elif 'm' in cpu:
        return float(cpu.rstrip('m'))
    else:
        return float(cpu) * (10**3)


def mem_float(memory):
    """
    Convert all memory values seen in K8s to Kilobytes.
    Kilobytes is taken because that is the standard of memory requests and limits in K8s
    :param memory: The memory value to be converted
    :return: Memory in Ki
    """
    if 'M' in memory:
        # Convert Megabytes to kilobytes
        return float(memory.rstrip('Mi').rstrip('M')) * (2 ** 10)
    elif 'G' in memory:
        return float(memory.rstrip('Gi').rstrip('G')) * (2 ** 20)
    else:
        return float(memory.rstrip('Ki'))
    

df["threads_per_second"] = df.apply(lambda x: int(round(x["thread_per_loop"]/x["ramp_time"])), axis=1)
df["request_cpu"] = df["request_cpu"].apply(cpu_float)
df["request_mem"] = df["request_mem"].apply(mem_float)
df["limit_cpu"] = df["limit_cpu"].apply(cpu_float)
df["limit_mem"] = df["limit_mem"].apply(mem_float)
df

/var/folders/31/zh2tg5jj2wsfynmj9mngmgg80000gn/T/ipykernel_62667/3214471151.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["threads_per_second"] = df.apply(lambda x: int(round(x["thread_per_loop"]/x["ramp_time"])), axis=1)
/var/folders/31/zh2tg5jj2wsfynmj9mngmgg80000gn/T/ipykernel_62667/3214471151.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["request_cpu"] = df["request_cpu"].apply(cpu_float)
/var/folders/31/zh2tg5jj2wsfynmj9mngmgg80000gn/T/ipykernel_62667/3214471151.py:38: SettingWithCo

,thread_per_loop,ramp_time,loops,instance_count,total_threads,request_cpu,request_mem,limit_cpu,limit_mem,min_per_node_cpu,...,min_cluster_mem,avg_cluster_mem,max_cluster_mem,error_rate,min_res_time,avg_res_time,max_res_time,received_kbytes,sent_kbytes,threads_per_second
3,20,10,10,1,200,2000.0,2097152.0,4000.0,4194304.0,0.008735,...,0.180148,0.185834,0.214941,0.0,43.0,76.340,132.0,980.225336,13764.434288,2
4,200,30,1,2,200,2000.0,2097152.0,4000.0,4194304.0,0.009014,...,0.179869,0.314572,0.416207,0.0,56.0,83.390,116.0,332.054843,4654.659555,7
7,10,60,10,1,100,2000.0,2097152.0,4000.0,4194304.0,0.008735,...,0.180179,0.186865,0.212863,0.0,42.0,63.020,103.0,90.469357,1270.380870,0
8,40,10,5,1,200,2000.0,2097152.0,4000.0,4194304.0,0.008735,...,0.180148,0.185838,0.214941,0.0,49.0,90.010,161.0,985.407575,13837.203865,4
10,200,30,1,2,200,1000.0,2097152.0,2000.0,4194304.0,0.009014,...,0.179869,0.313073,0.416207,0.0,58.0,82.235,113.0,332.054843,4654.659555,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,100,60,2,1,200,2000.0,2097152.0,4000.0,4194304.0,0.008735,...,0.180125,0.186963,0.214941,0.0,47.0,71.645,124.0,166.242075,2334.300879,2
378,50,60,10,1,500,500.0,1048576.0,1000.0,2097152.0,0.008735,...,0.180071,0.188834,0.214941,0.0,100.0,282.920,486.0,401.741184,5641.059423,1
384,100,5,1,1,100,2000.0,2097152.0,4000.0,4194304.0,0.009231,...,0.180179,0.182059,0.205072,0.0,76.0,187.220,314.0,1001.288358,14060.203603,20
386,40,30,5,2,200,2000.0,2097152.0,4000.0,4194304.0,0.009014,...,0.179869,0.317792,0.427889,0.0,42.0,60.170,116.0,336.115021,4711.574080,1


In [114]:
df_agg = df.groupby(['threads_per_second']).agg(
    {'request_cpu': 'min', 'request_mem': 'min',
     'limit_cpu': 'min', 'limit_mem': 'min', 'instance_count': 'max',
     'min_per_node_cpu': 'mean', 'avg_per_node_cpu': 'mean', 'max_per_node_cpu': 'mean',
     'min_per_node_mem': 'mean', 'avg_per_node_mem': 'mean', 'max_per_node_mem': 'mean',
     'min_cluster_mem': 'mean', 'avg_cluster_mem': 'mean', 'max_cluster_mem': 'mean',
     'min_res_time': 'mean', 'avg_res_time': 'mean', "max_res_time": 'mean',
     'received_kbytes': 'mean', 'sent_kbytes': 'mean'
    }
)
df_agg

,request_cpu,request_mem,limit_cpu,limit_mem,instance_count,min_per_node_cpu,avg_per_node_cpu,max_per_node_cpu,min_per_node_mem,avg_per_node_mem,max_per_node_mem,min_cluster_mem,avg_cluster_mem,max_cluster_mem,min_res_time,avg_res_time,max_res_time,received_kbytes,sent_kbytes
threads_per_second,,,,,,,,,,,,,,,,,,,
0,500.0,1048576.0,1000.0,2097152.0,2,0.008898,2.250268,53.960356,0.720173,0.746291,0.852905,0.180078,0.244040,0.314286,46.875000,81.150625,131.958333,131.597500,1846.285836
1,500.0,1048576.0,1000.0,2097152.0,2,0.008899,2.208566,54.935542,0.720151,0.744651,0.853104,0.180060,0.237807,0.306110,49.783784,91.619649,154.270270,271.195077,3805.130609
2,500.0,1048576.0,1000.0,2097152.0,2,0.008967,2.791291,59.285247,0.720155,0.745869,0.851190,0.180066,0.234210,0.309052,51.756757,103.234027,188.783784,432.695491,6069.918969
3,500.0,1048576.0,1000.0,2097152.0,2,0.008959,2.539250,57.066850,0.720113,0.746101,0.854330,0.180042,0.238976,0.304820,55.000000,82.031550,149.450000,281.868095,3954.531270
4,500.0,1048576.0,1000.0,2097152.0,2,0.009011,3.332771,71.070915,0.720106,0.747983,0.852258,0.180036,0.245744,0.342985,48.444444,106.500111,215.000000,997.781919,13993.065001
5,500.0,1048576.0,1000.0,2097152.0,2,0.009047,2.996663,63.302491,0.720063,0.743015,0.848204,0.180062,0.245661,0.343965,51.833333,103.425333,164.166667,770.212969,10800.711426
7,500.0,1048576.0,1000.0,2097152.0,2,0.008875,1.823303,59.192638,0.720038,0.744168,0.858246,0.180009,0.249827,0.315574,63.666667,92.284167,136.666667,331.555718,4651.696051
8,1000.0,2097152.0,2000.0,4194304.0,2,0.009046,3.656082,63.045531,0.720174,0.756473,0.855815,0.180062,0.213746,0.254358,52.400000,86.971800,208.000000,887.211349,12444.572313
10,500.0,1048576.0,1000.0,2097152.0,2,0.009118,3.809908,61.581952,0.720212,0.746273,0.846374,0.180073,0.229088,0.335371,61.571429,135.253214,220.357143,913.267242,12808.864855


In [115]:
df["total_threads"].unique()

array([ 200,  100,  500, 1000])